In [1]:
import fiona
import pandas as pd
import geopandas
import os

In [2]:
os.chdir('/Users/dannyha/SoCalEVCSNetwork')
os.getcwd()

'/Users/dannyha/SoCalEVCSNetwork'

# Data Importing and Cleaning

## SoCal Zipcodes and Geometry

In [101]:
socal_zipcodes = pd.read_csv (r'data/socal_zipcodes/socal_zip.csv')

In [102]:
socal_zipcodes = socal_zipcodes[['ZCTA5CE10','GEOID10','INTPTLAT10','INTPTLON10','geometry']]

In [103]:
socal_zipcodes = socal_zipcodes.rename(columns={"ZCTA5CE10": "Zip Code", "GEOID10": "GEOID", "INTPTLAT10": "Latitude",
                              "INTPTLON10": "Longitude", "geometry": "Geometry"})

In [104]:
socal_zipcodes.head()

,Zip Code,GEOID,Latitude,Longitude,Geometry
0,91504,691504,34.204568,-118.326365,"POLYGON ((-118.348583 34.208461, -118.348564 3..."
1,91606,691606,34.185767,-118.388324,"POLYGON ((-118.403066 34.19394, -118.402564 34..."
2,90640,690640,34.015444,-118.111012,"POLYGON ((-118.128344 33.985893, -118.127359 3..."
3,91770,691770,34.065493,-118.084529,"POLYGON ((-118.107291 34.066222, -118.107323 3..."
4,90290,690290,34.107728,-118.615981,"POLYGON ((-118.640641 34.125821, -118.640133 3..."


In [105]:
socal_zipcodes['Zip Code'] = socal_zipcodes['Zip Code'].apply(str)

In [106]:
type(socal_zipcodes['Zip Code'][0])

str

## LEHD Data

In [107]:
commuting = pd.read_csv (r'data/commuting/ca_od_aux_JT00_2018.csv')

In [108]:
commuting.columns = ['Workplace Census Block Code', 'Residence Census Block Code',
                    'Total number of jobs', 'Number of jobs of workers age 29 or younger',
                    'Number of jobs for workers age 30 to 54', 'Number of jobs for workers age 55 or older',
                    'Number of jobs with earnings $1250/month or less',
                     'Number of jobs with earnings $1251/month to $3333/month',
                    'Number of jobs with earnings greater than $3333/month',
                    'SI01','SI02','SI01','date']

In [109]:
commuting_filtered = commuting[['Workplace Census Block Code', 'Residence Census Block Code',
                    'Total number of jobs']]
commuting_filtered = commuting_filtered.reset_index().drop(columns=['index'])

In [111]:
commuting_filtered

,Workplace Census Block Code,Residence Census Block Code,Total number of jobs
0,60014001001007,250056451021050,1
1,60014001001024,132419702011062,1
2,60014002001001,480396615012022,1
3,60014002001016,410470015023004,1
4,60014002002003,450599210023035,1
...,...,...,...
179250,61150411003013,320310035071021,1
179251,61150411003021,50434902003002,1
179252,61150411003070,320050010001006,1
179253,61150411004037,320310022072004,1


In [112]:
commuting_filtered['Workplace Census Block Code'] = commuting_filtered['Workplace Census Block Code'].apply(str)
commuting_filtered['Residence Census Block Code'] = commuting_filtered['Residence Census Block Code'].apply(str)

In [113]:
commuting_filtered['Workplace Census Block Code'] = commuting_filtered['Workplace Census Block Code'].str.replace(
    r'.{4}$', '')
commuting_filtered['Residence Census Block Code'] = commuting_filtered['Residence Census Block Code'].str.replace(
    r'.{4}$', '')
commuting_filtered

,Workplace Census Block Code,Residence Census Block Code,Total number of jobs
0,6001400100,25005645102,1
1,6001400100,13241970201,1
2,6001400200,48039661501,1
3,6001400200,41047001502,1
4,6001400200,45059921002,1
...,...,...,...
179250,6115041100,32031003507,1
179251,6115041100,5043490200,1
179252,6115041100,32005001000,1
179253,6115041100,32031002207,1


In [114]:
tract_to_zip = pd.read_csv (r'data/ZIP_TRACT_032010.csv')
tract_to_zip

,ZIP,TRACT,RES_RATIO,BUS_RATIO,OTH_RATIO,TOT_RATIO
0,501,36103158607,0.000000,1.000000,0.000000,1.000000
1,544,36103158607,0.000000,1.000000,0.000000,1.000000
2,601,72001956300,0.004316,0.005063,0.000000,0.004310
3,601,72001956600,0.728760,0.448101,0.546667,0.703202
4,601,72001956700,0.266924,0.546835,0.453333,0.292488
...,...,...,...,...,...,...
145624,99925,2201000200,0.000000,0.000000,1.000000,1.000000
145625,99926,2201000400,0.000000,0.000000,1.000000,1.000000
145626,99927,2201000100,0.000000,0.000000,1.000000,1.000000
145627,99928,2130000100,0.000000,0.000000,1.000000,1.000000


In [115]:
type(tract_to_zip['TRACT'][0])

numpy.int64

In [116]:
tract_to_zip['ZIP'] = tract_to_zip['ZIP'].apply(str)

In [117]:
socal_zips = tract_to_zip[tract_to_zip['ZIP'].isin(socal_zipcodes['Zip Code'])]

In [118]:
socal_zips.head()

,ZIP,TRACT,RES_RATIO,BUS_RATIO,OTH_RATIO,TOT_RATIO
127911,90001,6037239200,0.000000,0.102041,0.025126,0.011305
127912,90001,6037239330,0.000000,0.060587,0.020101,0.006849
127913,90001,6037239500,0.094094,0.036990,0.025126,0.086315
127914,90001,6037239800,0.100367,0.028061,0.140704,0.093896
127915,90001,6037240010,0.000918,0.014031,0.005025,0.002394


In [119]:
socal_zips['TRACT'] = socal_zips['TRACT'].apply(str)

<ipython-input-119-05d622e1dd8c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  socal_zips['TRACT'] = socal_zips['TRACT'].apply(str)


In [120]:
commuting_filtered_socal = commuting_filtered[commuting_filtered['Workplace Census Block Code'].isin(
    socal_zips['TRACT'])]
commuting_filtered_socal

,Workplace Census Block Code,Residence Census Block Code,Total number of jobs
27391,6037101210,4001944901,1
27392,6037101210,4013104202,1
27393,6037101220,51770000601,1
27394,6037101220,12099004815,1
27395,6037101220,17031801701,1
...,...,...,...
96501,6059110606,37007920100,1
96502,6059110606,37125951100,1
96503,6059110607,4013061041,1
96504,6059110607,4013082205,1


In [96]:
commuting_filtered_socal = commuting_filtered_socal.groupby(
    ['Residence Census Block Code','Workplace Census Block Code']).sum()

In [99]:
commuting_filtered_socal.sort_values('Total number of jobs',ascending=False)

Total number of jobs
Residence Census Block Code Workplace Census Block Code                      
36061017900                 6037311600                                     10
36061015900                 6037311700                                      8
36061013900                 6037311600                                      8
36061018300                 6037311700                                      8
36061015700                 6037311600                                      8
...                                                                       ...
32031003108                 6059042312                                      1
32031003109                 6037113321                                      1
                            6059062612                                      1
32031003110                 6037113421                                      1
9015815000                  6037191710                                      1

[39046 rows x 1 columns]

# Block

In [22]:
socal_zips['TRACT'] = socal_zips['TRACT'].apply(int)

<ipython-input-22-a85bbcb9a734>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  socal_zips['TRACT'] = socal_zips['TRACT'].apply(int)


In [23]:
#RES_RATIO = The ratio of residential addresses in the ZIP – Tract, County, or CBSA part 
#to the total number of residential addresses in the entire ZIP.

socal_zips_cleaned = socal_zips[['TRACT','ZIP','RES_RATIO']]

In [24]:
#converting zip_and_tract to dictionary
zip_and_tract_dict = socal_zips_cleaned.set_index('TRACT').T.to_dict('list')

<ipython-input-24-23ca5f4ecff8>:2: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  zip_and_tract_dict = socal_zips_cleaned.set_index('TRACT').T.to_dict('list')


In [25]:
#remove last four digit of block code to make is census tract code
commuting_filtered['Workplace Census Block Code'] = commuting_filtered['Workplace Census Block Code'].apply(str)
commuting_filtered['Residence Census Block Code'] = commuting_filtered['Residence Census Block Code'].apply(str)
commuting_filtered['Workplace Census Block Code'] = commuting_filtered['Workplace Census Block Code'].str.replace(r'\d{4}$', '')
commuting_filtered['Residence Census Block Code'] = commuting_filtered['Residence Census Block Code'].str.replace(r'\d{4}$', '')

In [26]:
commuting_filtered.columns = ['Workplace Census Tract', 'Residence Census Tract','Total Jobs']

In [100]:
#changing Total Jobs column to integers
commuting_filtered['Total Jobs'] = commuting_filtered['Total Jobs'].apply(int)

KeyError: 'Total Jobs'

In [28]:
# removing insignificant rows
commuting_filtered = commuting_filtered[commuting_filtered['Total Jobs'] > 10]

In [29]:
socal_zips_cleaned['TRACT'] = socal_zips_cleaned['TRACT'].apply(int)

<ipython-input-29-39e02b299d53>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  socal_zips_cleaned['TRACT'] = socal_zips_cleaned['TRACT'].apply(int)


In [30]:
commuting_filtered['TRACT'] = commuting_filtered['Residence Census Tract']

In [31]:
commuting_filtered.head()

,Workplace Census Tract,Residence Census Tract,Total Jobs,TRACT
10125,6015000201,41015950400,12,41015950400


In [1081]:
socal_zips_dict = socal_zips_cleaned.set_index('TRACT').T.to_dict('list')
socal_zips_dict = {int(key):socal_zips_dict[key] for key in socal_zips_dict}

<ipython-input-1081-0094f37d3faa>:1: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  socal_zips_dict = socal_zips_cleaned.set_index('TRACT').T.to_dict('list')


In [1082]:
commuting_filtered['Workplace Census Tract'] = commuting_filtered['Workplace Census Tract'].apply(int)

In [1083]:
commuting_grouped = commuting_filtered.groupby(
    ['Residence Census Tract','Workplace Census Tract']).sum('Total Jobs')
commuting_grouped

Total Jobs
Residence Census Tract Workplace Census Tract            
6001408000             6001408000                      12
6001420300             6001422300                      27
                       6001422600                      45
                       6075017700                      28
6001420400             6001422300                      14
...                                                   ...
6113011206             6095253300                     109
                       6113010800                      12
                       6113011500                      39
6113011300             6095253300                      19
                       6113011300                      37

[2787 rows x 1 columns]

In [1085]:
commuting_grouped.sort_values('Total Jobs',ascending=False).head(20)

,,Total Jobs
Residence Census Tract,Workplace Census Tract,
6059062614,6059062614,1355
6053014105,6053014104,374
6071007302,6071007303,369
6081610202,6081611700,355
6083002930,6083002922,343
6073008363,6073008305,317
6051000200,6051000200,271
6075061500,6075061500,263
6085505008,6085505008,263


In [720]:
ev_geo = zipcodes.merge(ev_percentage,on='Zip Code')

In [723]:
ev_geo['ev_percentage'] = ev_geo['Electric Vehicles'] / ev_geo['Total Vehicles']

In [726]:
ev_geo

,Zip Code,geometry,index,Electric Vehicles,Total Vehicles,ev_percentage
0,91504,"POLYGON ((-118.34858 34.20846, -118.34856 34.2...",236,894,42066.0,0.021252
1,91606,"POLYGON ((-118.40307 34.19394, -118.40256 34.1...",243,736,61560.0,0.011956
2,90640,"POLYGON ((-118.12834 33.98589, -118.12736 33.9...",129,842,98642.0,0.008536
3,91770,"POLYGON ((-118.10729 34.06622, -118.10732 34.0...",261,1484,101572.0,0.014610
4,90290,"POLYGON ((-118.64064 34.12582, -118.64013 34.1...",97,710,11192.0,0.063438
...,...,...,...,...,...,...
346,92694,"POLYGON ((-117.65254 33.56291, -117.65168 33.5...",309,3014,48286.0,0.062420
347,92831,"POLYGON ((-117.91359 33.86940, -117.91359 33.8...",328,1138,49684.0,0.022905
348,92707,"MULTIPOLYGON (((-117.87814 33.66436, -117.8780...",315,612,87650.0,0.006982
349,92648,"MULTIPOLYGON (((-118.01199 33.69955, -118.0120...",290,2830,80342.0,0.035224


In [727]:
ev_geo.to_file("socal_ev.shp")

<ipython-input-727-e99afb744cc3>:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  ev_geo.to_file("socal_ev.shp")
